# Solução Exercício Python Geo (Kognitalabs)
-> A resolução dos exercícios a seguir considera que os dados se referem a uma rede de varejo do segmento alimentício (doces e chocolates). A análise tem como objetivo proporcionar uma compreensão estratégica e fornecer insights que apoiarão o cliente no entendimento geoespacial das unidades. Além disso, visa oferecer informações para auxiliar na expansão das unidades da rede.

In [ ]:
#bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
from geopy.geocoders import Nominatim
from scipy.stats import linregress



In [ ]:
#carregar e mostrar o arquivo
arq_poi = pd.read_parquet("pontos_interesse_geograficos.parquet")
arq_poi

In [ ]:
#VERIFICAÇÕES
arq_poi.isna().sum()


In [ ]:
arq_poi.isnull().sum()

In [ ]:
arq_poi["tipo_negocio"].describe()

In [ ]:
negocios = arq_poi.groupby("tipo_negocio").size()
negocios

•**Tarefa 1:**

    Determinar os pontos de interesse geográficos (POIs) localizados dentro de cada município, e mapear as categorias relacionadas a potenciais concorrentes.

In [ ]:
# TRANSFORMAR AS COORDENADAS GEOGRAFICAS PARA DESCOBRIR A O MUNICIPIO(endereco)
# Inicial
geolocator = Nominatim(user_agent="meu_app")

enderecos = []

# Infelizmente só consegui processar as 1000 primeiras linhas do DataFrame.
arq_poi = arq_poi.head(1000)

# percorrer as linhas do Dataset
for index, row in arq_poi.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    location = geolocator.reverse((latitude, longitude))
    endereco_completo = location.address if location else "Endereço não encontrado"

    # Extrair  o endereço
    endereco_partes = endereco_completo.split(',')
    if len(endereco_partes) > 5:
        endereco_resumido = endereco_partes[5].strip()
    else:
        endereco_resumido = "Dados insuficientes"

    enderecos.append(endereco_resumido)
    print(f'{endereco_resumido}')

# criar uma nova coluna no dataset com os endereços
arq_poi['endereco'] = enderecos

print("Processamento concluído.")


In [ ]:

arq_poi

In [ ]:
#PONTOS DE INTERESSE X CONCORRENTES
poi = ['academia','aeroporto','amenidade', 'autoescola', 'banco', 'beleza', 'bombeiro', 'caixa', 'cartorio', 'cinema', 'clinica', 'clube', 'combustivel', 'concessionaria', 'correio', 'dentista', 'distribuidor', 'escola', 'escritório', 'esporte', 'estacionamento', 'evento', 'eventos', 'exercito', 'fabricantes', 'farmacia', 'fazenda', 'fornecedor', 'heliporto', 'hospedagem', 'hospital', 'igreja', 'laboratorio', 'medico', 'moradia', 'oficina', 'organizacao', 'outros', 'petshop', 'policia', 'praia publica', 'prestador', 'publico', 'saude', 'servicopublico', 'transporte', 'universidade', 'veterinario' ]
concorrentes = ['comercio', 'negocio', 'padaria', 'restaurante', 'shopping', 'supermercado' ]

In [ ]:
# Filtrar e agrupar Pontos de Interesse geográficos(POIs)
fil_poi = arq_poi[arq_poi['tipo_negocio'].isin(poi)]
g1 = fil_poi.groupby("tipo_negocio")['endereco'].size().reset_index(name='counts') # agrupado por negócio


# Mapear as categorias de Potenciais Concorrentes
fil_concorrentes = arq_poi[arq_poi['tipo_negocio'].isin(concorrentes)]
g1_concorrentes = fil_concorrentes.groupby("tipo_negocio")['endereco'].size().reset_index(name='counts') # agrupado por negócio


•**Tarefa 2:**

    Plotar mapa quantidade total de POIs por município.

In [ ]:
#Grafico de barras
plt.figure(figsize=(16, 6))
sns.barplot(data=g1, y="counts", x="tipo_negocio").set(title="Contagem de Negócios por Tipo")
plt.xticks(rotation=80)
plt.show()

•**Tarefa 3:**

    Para cada uma das top 5 categorias de POIs (em termos de quantidade), fazer o mesmo que o item 2.

In [ ]:
#Mostrar quem são o top 5
top5 = g1.sort_values(by='counts', ascending=False)
top5.head(5)

In [ ]:
# TOP 1 - categoria 'prestador'
categoria1 = arq_poi[arq_poi['tipo_negocio'] == 'prestador']

# Agrupar por município e contar a quantidade
categoria1_counts = categoria1.groupby('endereco').size().reset_index(name='counts')

# Ordenar os dados por quantidade para melhor visualização
categoria1_counts = categoria1_counts.sort_values(by='counts', ascending=False)

# grafico
plt.figure(figsize=(16, 8))
sns.barplot(data=categoria1_counts, 
            x='endereco', 
            y='counts', 
            palette="viridis")
plt.xticks(rotation=90)
plt.title("Quantidade de POIs Prestadores por Município")
plt.xlabel('Município')
plt.ylabel('Quantidade de Prestadores')
plt.show()

In [ ]:
# TOP 2 - categoria 'beleza'
categoria2 = arq_poi[arq_poi['tipo_negocio'] == 'beleza']

# Agrupar por município e contar a quantidade
categoria2_counts = categoria2.groupby('endereco').size().reset_index(name='counts')

# Ordenar os dados por quantidade para melhor visualização
categoria2_counts = categoria2_counts.sort_values(by='counts', ascending=False)

# grafico
plt.figure(figsize=(16, 8))
sns.barplot(data=categoria2_counts, 
            x='endereco', 
            y='counts', 
            palette="viridis")
plt.xticks(rotation=90)
plt.title("Quantidade de POIs beleza por Município")
plt.xlabel('Município')
plt.ylabel('Quantidade de POIs beleza')
plt.show()

In [ ]:
# TOP 3 - categoria 'escola'
categoria3 = arq_poi[arq_poi['tipo_negocio'] == 'escola']

# Agrupar por município e contar a quantidade
categoria3_counts = categoria3.groupby('endereco').size().reset_index(name='counts')

# Ordenar os dados por quantidade para melhor visualização
categoria3_counts = categoria3_counts.sort_values(by='counts', ascending=False)

# grafico
plt.figure(figsize=(16, 8))
sns.barplot(data=categoria3_counts, 
            x='endereco', 
            y='counts', 
            palette="viridis")
plt.xticks(rotation=90)
plt.title("Quantidade de POIs escola por Município")
plt.xlabel('Município')
plt.ylabel('Quantidade de POIs escola')
plt.show()

In [ ]:
# TOP 4 - categoria 'igreja'
categoria4 = arq_poi[arq_poi['tipo_negocio'] == 'igreja']

# Agrupar por município e contar a quantidade
categoria4_counts = categoria4.groupby('endereco').size().reset_index(name='counts')

# Ordenar os dados por quantidade para melhor visualização
categoria4_counts = categoria4_counts.sort_values(by='counts', ascending=False)

# grafico
plt.figure(figsize=(16, 8))
sns.barplot(data=categoria4_counts, 
            x='endereco', 
            y='counts', 
            palette="viridis")
plt.xticks(rotation=90)
plt.title("Quantidade de POIs igreja por Município")
plt.xlabel('Município')
plt.ylabel('Quantidade de POIs igreja')
plt.show()

In [ ]:
# TOP 5 - categoria 'oficina'
categoria5 = arq_poi[arq_poi['tipo_negocio'] == 'oficina']

# Agrupar por município e contar a quantidade
categoria5_counts = categoria5.groupby('endereco').size().reset_index(name='counts')

# Ordenar os dados por quantidade para melhor visualização
categoria5_counts = categoria5_counts.sort_values(by='counts', ascending=False)

# grafico
plt.figure(figsize=(16, 8))
sns.barplot(data=categoria5_counts, 
            x='endereco', 
            y='counts', 
            palette="viridis")
plt.xticks(rotation=90)
plt.title("Quantidade de POIs oficina por Município")
plt.xlabel('Município')
plt.ylabel('Quantidade de POIs oficina')
plt.show()

•**Tarefa 4:**

    Para as categorias de POIs que foram identificadas como possíveis concorrentes, fazer o mesmo que o item 2.

In [ ]:
#Grafico de barras
plt.figure(figsize=(16, 6))
sns.barplot(data=g1_concorrentes, 
            y="counts", 
            x="tipo_negocio").set(title="Contagem de Negócios Concorrentes por Tipo")
plt.xticks(rotation=80)
plt.show()

•**Tarefa 5:**

    Para os dados utilizados para gerar os 7 mapas acima, determinar a correlação espacial (coeficiente I de Moran global) da quantidade de POIs no nível de município. 

•**Tarefa 6:**

    Plotar os dados de faturamento (considere a mediana do faturamento no período disponibilizado) exibindo informações de lat long e o faturamento no "hover" do mouse sobre o ponto.

•**Tarefa 7:**

    Baseado no faturamento histórico, determinar programaticamente quais unidades estão com tendência de alta ou de baixa no faturamento. Adicionar essa informação de alta ou baixa no gráfico do item 6.

In [ ]:
#Carregar e mostrar o arquivo
arq_fat = pd.read_parquet("unidades_faturamento.parquet")
arq_fat

In [ ]:

# Determinar com base nas colunas faturamento e ID unidade
faturamento_cols = arq_fat.columns[1:-3]
id_col = 'id_unidade'

# Transformar os dados para que cada linha seja uma unidade e cada coluna um mês
arq_fat_long = arq_fat.melt(id_vars=[id_col], value_vars=faturamento_cols, var_name='data', value_name='faturamento')

# Ordenar por unidade e data
arq_fat_long = arq_fat_long.sort_values(by=[id_col, 'data'])

# Função para calcular a tendência de faturamento
def calcular_tendencia(df):
    x = np.arange(len(df))
    y = df['faturamento'].values
    slope, _, _, _, _ = linregress(x, y)
    
    # Determinar se a tendência é alta ou baixa
    if slope > 0:
        return 'alta'
    elif slope < 0:
        return 'baixa'
    else:
        return 'estável'

# Aplicar a função para cada unidade e adicionar a nova coluna
arq_fat_tendencia = arq_fat_long.groupby(id_col).apply(calcular_tendencia).reset_index(name='tendencia_faturamento')

# Adicionar a coluna de tendência ao DataFrame original
arq_fat = arq_fat.merge(arq_fat_tendencia, on=id_col, how='left')


•**Tarefa 8:**

    Plotar o faturamento médio por município

In [ ]:
# TRANSFORMAR AS COORDENADAS GEOGRAFICAS PARA DESCOBRIR A O MUNICIPIO(endereco)
# Inicial
geolocator = Nominatim(user_agent="meu_app")

enderecos = []

# percorrer as linhas do Dataset
for index, row in arq_fat.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    location = geolocator.reverse((latitude, longitude))
    endereco_completo = location.address if location else "Endereço não encontrado"

    # Extrair  o endereço
    endereco_partes = endereco_completo.split(',')
    if len(endereco_partes) > 5:
        endereco_resumido = endereco_partes[5].strip()
    else:
        endereco_resumido = "Dados insuficientes"

    enderecos.append(endereco_resumido)
    print(f'{endereco_resumido}')

# criar uma nova coluna no dataset com os endereços
arq_fat['endereco'] = enderecos

print("Processamento concluído.")


In [ ]:
# Selecionar apenas as colunas de faturamento, excluindo 'id_unidade', 'latitude', 'longitude' e 'endereco'
faturamento_cols = arq_fat.columns[1:-3]  # Ajuste conforme a estrutura do seu DataFrame

# Somar os valores de faturamento para cada linha
arq_fat['faturamento_total'] = arq_fat[faturamento_cols].sum(axis=1)

# Calcular a média dos faturamentos por endereço (município)
faturamento_medio = arq_fat.groupby('endereco')['faturamento_total'].mean().reset_index()

# Ordenar por faturamento médio em ordem decrescente
faturamento_medio = faturamento_medio.sort_values(by='faturamento_total', ascending=False)

# Criar o gráfico de barras
plt.figure(figsize=(15, 10))
plt.barh(faturamento_medio['endereco'], faturamento_medio['faturamento_total'], color='teal')
plt.xlabel('Faturamento Médio')
plt.ylabel('Município')
plt.title('Faturamento Médio por Município')
plt.gca().invert_yaxis()  # Inverter o eixo y para mostrar o município com maior faturamento no topo
plt.show()


•**Tarefa 9:**

    Idem para o faturamento médio por 100 mil habitantes.

•**Tarefa 10:**

    Dada a tabela de faturamentos fictícios, determinar a correlação espacial (coeficiente I de Moran global) do faturamento total no nível de município

•**Tarefa 11:**

    Dadas as análises elaboradas, que conclusões consegue extrair do estudo? Caso seja útil, fique à vontade para explorar outros tipos de análises adicionais.

Com base nas análises realizadas, foi possível distinguir entre pontos de interesse geográficos (POIs) e pontos com potenciais concorrentes. Além disso, identificamos quais tipos de negócios predominam em locais específicos e quais unidades apresentam o melhor desempenho financeiro por município. Essas informações podem ajudar a identificar áreas atrativas para a expansão das unidades da rede varejista, auxiliando nas decisões de planejamento urbano e expansão de negócios. Assim, a empresa pode utilizar essas análises para ajustar suas estratégias operacionais com base em POIs, concorrentes, desempenhos históricos e municípios.

Do ponto de vista estratégico e geográfico, áreas com um alto número de setores de negócios favoráveis (como escolas e igrejas) podem indicar mercados mais maduros ou com maior demanda e, portanto, podem ser alvos prioritários para a abertura de novas unidades. Em contrapartida, a análise dos setores de negócios concorrentes pode revelar quais áreas estão saturadas e onde há maior competição, orientando a escolha dos locais para novas unidades.

No aspecto financeiro, conseguimos avaliar o desempenho das diferentes unidades para identificar padrões de baixa no faturamento que merecem uma análise mais aprofundada, assim como padrões de sucesso que podem ser replicados em outras áreas.